# Maximum Likelihood Estimation in TensorFlow

# 2. Optimizers

## 2.4 Exercises

### 2.4.3 Height and weight

Dataset contains 101 observations of people's weights and heights. The relationship between weight and height can be expressed by equation $weight = a \cdot height ^ b$, where $a$ and $b$ are parameters. The task is to estimate these parameters.

In [ ]:
library(tidyverse)
library(tensorflow)
library(plotly)

In [ ]:
weight_height <- readr::read_csv('data/weight_height.csv') # read data

In [ ]:
# look at the data
weight_height %>% 
  plot_ly() %>% 
  add_markers(x = ~h, y = ~w) %>%
  layout(xaxis = list(title = 'Height'), 
         yaxis = list(title = 'Weight'))

In [ ]:
# express data as nodes
ta <- NA # TODO: create a variable for a. Initialise it with 1.0.
tb <- NA # TODO: create a variable for b. Initialise it with 1.0.

th <- NA # TODO: create a constant for height. 
tw <- NA # TODO: create a constant for weight. 

w_ <- NA # TODO: calculate weight according to the equation in instruction


In [ ]:
# prepare model 
loss <- NA # TODO: from losses choose MSE as loss function, 
           # set appropriate: labels (observed data) and predictions (estimations)
optimizer <- NA # TODO: from train choose Adam Optimizer and set learning rate= 0.05

train <- NA #TODO: minimize loss in a given optimizer

In [ ]:
sess = tf$Session() # run session 
sess$run(tf$global_variables_initializer()) # init variable

In [ ]:
iter = 10000 # number of iterations to run

# table for storing model parameter optimisation progress log
r = tibble(
  it = 1:iter,
  loss = rep(NA_real_, iter),
  a = rep(NA_real_, iter),
  b = rep(NA_real_, iter)
)


In [ ]:
# train
for(i in 1:iter){
  NA # TODO: run one iteration of training 
  r$loss[i] = sess$run(loss)
  r$a[i] = sess$run(ta)
  r$b[i] = sess$run(tb)
}

Let's visualise results

In [ ]:
plot_ly(data = r) %>%
  add_lines(
    x = ~it,
    y = ~loss,
    name = 'loss'
  )

In [ ]:
plot_ly(data = r) %>%
  add_lines(
    x = ~it,
    y = ~a,
    name = 'a'
  ) %>%
  add_lines(
    x = ~it,
    y = ~b,
    name = 'b'
  )

# 3. Likelihood

<p>
    <b>Likelihood</b> measures the plausibility of a model parameter value, given observations we have.<br/>
    $\mathcal{L}(\theta \mid x) = p_\theta (x) = P (X=x; \theta)$, where
    <ul>
        <li>$\mathcal{L}(\theta \mid x)$ is the likelihood value</li>
        <li>$p_\theta (x) = P (X=x; \theta)$ is the probability of observing $x$ given parameter values are equal $\theta$</li>
        <li>$x$ observed data</li>
        <li>$\theta$ parameter values</li>
    </ul>
</p>
<p>
    In other words, it tells us what is the probability of observing what we observed, assuming the given model parameter values. The value of this probability changes depending on the $\theta$. When these parameters have values that do not let $x$ occur, $P (X=x; \theta) = 0$, and when it has values in which $x$ must always occur $P (X=x; \theta) = 1$.
</p>
<p>
    Of course, we usually have more than one observation: $x_1$, $x_2$, $x_3$, ... So its likelihood is equal to:<br/>
    $\mathcal{L}(\theta \mid x_1, x_2, ..., x_n) = P (X=[x_1, x_2, ..., x_n]; \theta) = \prod_{i=1}^n P (X=x_i; \theta)$<br/>
</p>
<p>
    The $\theta$ for which the value of $\prod_{i=1}^n P (X=x_i; \theta)$ is highest is called the Maximum Likelihood Estimate, and in the frequentist statistics it is assumed to be an accurate estimation of a latent parameter (a parameter that can't be measured directly).
</p>

## 3.1 Likelihood exercise

You tossed a coin three times. You observed heads once and tails two times. What are the likelihoods that the probability of the coin landing heads up is equal to:
<ul>
    <li>$1 \over 2$</li>
    <li>$1 \over 3$</li>
    <li>$1 \over 5$</li>
    <li>$11 \over 30$</li>
    <li>$9 \over 30$</li>
</ul>
What do you think, which of these probabilities is the most likely assuming you don't know what probability you should expect?

## 3.2 Cross-entropy and MLE

<p>
    Now, let's take the equation from earlier<br/>
    $\mathcal{L}(\theta \mid x_1, x_2, ..., x_n) = \prod_{i=1}^n P (X=x_i; \theta)$
</p>
<p>
    and logarithmise both sides of this equation:<br/>
    $log \mathcal{L}(\theta \mid x_1, x_2, ..., x_n) = log\prod_{i=1}^n P (X=x_i; \theta) = \sum_{i=1}^n log P (X=x_i; \theta)$<br/>
    logarithm of the likelihood is called the log-likelihood and has the same maximum as the likelihood because logarithm is a monotonically increasing function.
</p>
<p>
    Now, let's assume that we have a probability that something happened ($y_i = 1$), e.g. a student answered a test question correctly $P (Y=1; \theta)$, then the probability of that not occuring ($y_i = 0$) in the same conditions - $P (Y=0; \theta)$ - is equal to: $1 - P (Y=1; \theta)$. Then:<br/>
    $log \mathcal{L}(\theta \mid y_i) = log P (Y=y_i; \theta) = y_i \cdot log P(Y=1; \theta) + (1 - y_i) \cdot log (1 - P(Y=1; \theta))$
</p>
<p>
    For simplification, let's denote $P(Y=1; \theta)$ for given $y_i$ as $p_{i,\theta}$. Now:<br/>
    $log \mathcal{L}(\theta \mid y_i) = y_i \cdot log p_{i,\theta} + (1 - y_i) \cdot log (1 - p_{i,\theta})$
</p>
<p>
    And now let's multiply both sides by $-1$:<br/>
    $-log \mathcal{L}(\theta \mid y_i) = -(y_i \cdot log p_{i,\theta} + (1 - y_i) \cdot log (1 - p_{i,\theta})) = cross$-$entropy(p_{i,\theta}, y_i)$
</p>
<p>
    Thus:
    <ul>
        <li><b>when we minimise the value of cross-entropy we also maximise the likelihood</b> (because we minimise the negative log-likelihood)</li>
        <li>what we do in TensorFlow is not magic that just works, it's a well established statistical method that has been in use for over a century</li>
        <li>it also applies to very complicated models called neural networks. Just in this case, we do not believe that parameters we estimate make any sense</li>
    </ul>
</p>

## 3.3 Example - an unbalanced coin and MLE

In [ ]:
# Loading packages we will need
library(tensorflow)
library(tidyverse)
library(plotly)

In this section we will use TF and MLE to estimate the probability that an unbalanced coin lands heads up - $p_H$. Below we have a result of 100 coin tosses.

In [ ]:
# 1 - HEADS, 0 - TAILS
results = c(0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1)

In [ ]:
t_p = tf$Variable(0.01) # the variable containing a probability of a coin landing heads up. Let's initialise
                        # it with 0.01 (assume this coin almost always lands tails up)
t_y = tf$constant(results)

p_vector = tf$fill(list(tf$size(t_y)), t_p) # we need to calculate the expected probability for each coin toss we observed
                                            # however, it's the same in all attempts, so we just assume the current t_p estimate for each observation

t_loss = tf$losses$log_loss(t_y, p_vector) # now we need to calculate the cross-entropy between estimated probability, and each observation

# and now, we also need to establish the optimisation process...
optimizer = tf$train$AdamOptimizer(0.1)
train = optimizer$minimize(t_loss)

# as always we need to create a session
sess = tf$Session() 
sess$run(tf$global_variables_initializer())

In [ ]:
iter = 1000 # number of iterations to run

# table for storing model parameter optimisation progress log
r = tibble(
    it = 1:iter,
    loss = rep(NA_real_, iter),
    prob = rep(NA_real_, iter)
)

In [ ]:
# now we've got everything, so LET'S START THE ESTIMATION
for(i in 1:iter){
    sess$run(train)
    r$loss[i] = sess$run(t_loss)
    r$prob[i] = sess$run(t_p)
}

In [ ]:
cat('Final probability that the coin lands heads up is ', sess$run(t_p), '.')

When we finished the estimation process, we can check how the loss value and the estimated probability was changing during the iteration:

In [ ]:
plot_ly() %>%
  add_lines(
    x = ~it,
    y = ~loss,
    data = r
  )

In [ ]:
plot_ly() %>%
  add_lines(
    x = ~it,
    y = ~prob,
    data = r
  )

## 3.4 Example - MLE of normal distribution parameters

<p>
    In this section, we will do the maximum likelihood estimation of parameters for a continuous probability distribution. In such cases we need to use an alternative likelihood definition:<br/>
    $\mathcal{L}(\theta \mid x) = f (X=x; \theta)$<br/>
    where $f (X=x; \theta)$ is the value of the probability density function (<i>pdf</i>)
</p>
<p>
    Since we are working with the normal distribution, which have two parameters (mean and standard deviation), the total likelihood of all observations takes form:
    $\mathcal{L}(mean, sd \mid x_1, x_2, ..., x_n) = \prod_{i=1}^n f (X=x_i; mean, sd)$,
    and<br/>
    $log\mathcal{L}(mean, sd \mid x_1, x_2, ..., x_n) = \sum_{i=1}^n log f (X=x_i; mean, sd)$
</p>
<p>
    <b>QUESTION</b>: is there any reason why we should prefer one of these forms over another? (e.g. a sum of logarithms over a product)
</p>

In [ ]:
# Loading packages we will need
library(tensorflow)
library(tidyverse)
library(plotly)

Let's start with generating some sample data:

In [ ]:
nd_mean = 3   # the mean of our distribution
nd_sd = 2     # the standard deviation of our distribution
nd_data = rnorm(10000, nd_mean, nd_sd) # and the actual data

Now, we need to build a model:

In [ ]:
t_mean = tf$Variable(0.0) # a variable for the mean of the distribution, let's set it initially to 0
t_sd = tf$Variable(1.0)   # a variable for the sd of the distribution, let's set it initially to 1

dist = tf$distributions$Normal(loc = t_mean, scale = t_sd) # a distribution we will use, it takes variables storing mean and sd as parameters

lprobs = dist$log_prob(nd_data) # log-likelihoods of parameters for each observation, i.e. logarithms of pdf values assuming current t_mean and t_sd 
negLL = -tf$reduce_sum(lprobs)  # sum of observations likelihoods. It's negative, because it is a value we will be minimising 

# of course, we also need to establish the optimisation process...
optimizer = tf$train$AdamOptimizer(0.1)
train = optimizer$minimize(negLL)

# ... and create a session
sess = tf$Session() 
sess$run(tf$global_variables_initializer())

In [ ]:
iter = 1000 # number of iterations to run

# table for storing model parameter optimisation progress log
r = tibble(
    it = 1:iter,
    negLL = rep(NA_real_, iter),
    mean = rep(NA_real_, iter),
    sd = rep(NA_real_, iter)
)

In [ ]:
# now we've got everything, so let's start
for(i in 1:iter){
    sess$run(train)
    r$negLL[i] = sess$run(negLL)
    r$mean[i] = sess$run(t_mean)
    r$sd[i] = sess$run(t_sd)
}

Now when our model parameters were fitted to the data, we can check how it went. Let's start with the likelihood maximisation (or rather minimisation of the negative log-likelihood):

In [ ]:
plot_ly() %>%
  add_lines(
    x = ~it,
    y = ~negLL,
    data = r
  )

We can also check how values of mean and standard deviation were changing during iteration:

In [ ]:
plot_ly(data = r) %>%
  add_lines(
    x = ~it,
    y = ~mean,
    name = 'mean'
  ) %>%
  add_lines(
    x = ~it,
    y = ~sd,
    name = 'sd'
  )

## 3.5 Probability estimation

<p>
    OK, so now we know how we can measure how accurate is our probability estimation. Now, we only need to have a way to estimate the probability, i.e. a value that isn't lower than 0, and not higher than 1. We can use the log-odds (logit) and the logistic function for that purpose.
</p>
<p>
    $\operatorname{log-odds}(p)=\operatorname{logit}(p)=\log\left( \frac{p}{1-p} \right)$<br/>
    It converts a value from the interval $[0;1]$ to the interval $[-∞;+∞]$. And now, it's something that we can easily estimate using e.g. linear regression. Later, we just need to convert it back to a probability using a sigmoid function:<br/>
</p>
<p>
    $p = \operatorname{sigmoid}(logit) = \frac{1}{1 + e^{-logit}}$
</p>

### 3.5.1 Example - linear logistic regression

<p>
    In this example we will use MLE to estimate parameters $a$, $b$ and $c$ of the logistic linear regression model. <br/>
</p>
<p>
    $\operatorname{logit}(z = 1) = a \cdot x + b \cdot y + c $
</p>
<p>
    $\operatorname{P}(z = 1) = \frac{1}{1 + e^{-\operatorname{logit}(z = 1)}}$
</p>

In [ ]:
library(tidyverse)
library(tensorflow)

In [ ]:
example_351 = read_csv('data/example_351.csv')
head(example_351)

In [ ]:
# loading data into the graph
t_x = tf$constant(example_351$x, dtype = 'float32')
t_y = tf$constant(example_351$y, dtype = 'float32')
t_z = tf$constant(example_351$z, dtype = 'float32')

In [ ]:
# creation of variables storing parameter values
t_a = tf$Variable(42) # as we know, this is a really good number
t_b = tf$Variable(42)
t_c = tf$Variable(42)

In [ ]:
t_logit = t_a * t_x + t_b * t_y + t_c  # log-odds of P(z = 1)
t_z_ = tf$sigmoid(t_logit) # and the actual probability of z = 1

In [ ]:
# now we need to create an optimiser, calculate loss, and minimise its value
optimizer = tf$train$AdamOptimizer(learning_rate = 0.1)
loss = tf$losses$log_loss(t_z, t_z_)
train = optimizer$minimize(loss)

sess = tf$Session()
sess$run(tf$global_variables_initializer())

In [ ]:
# now we need to train model parameters
for(i in 0:2000){
    sess$run(train)
    if(i %% 200 == 0){
        cat(i, ') ', format(Sys.time(), "%Y-%m-%d %X"), ' Loss: ', sess$run(loss), '\n', sep = '')
    }
}

In [ ]:
# and now we can check what are the final parameter values
cat("t_a: ", sess$run(t_a), "\n")
cat("t_b: ", sess$run(t_b), "\n")
cat("t_c: ", sess$run(t_c), "\n")

## 3.6 Exercise - A bribe-taking mayor

In [ ]:
library(tidyverse)
library(tensorflow)

<p>
    The mayor of Los Data-flowos is a well-known bribe-taker. The local data science club decided to find out in what conditions he is eager to take a bribe. They discovered two main factors that influence it.
</p>
<p>
    The first one is the amount of money offered, which have a logarhitmic influence on the mayor, which is proportional to:<br/>
    $mc \cdot log(ao)$, where<br/>
    <ul>
        <li><b>mc</b> - money coefficient, currently unknown and needs to be estimated</li>
        <li><b>ao</b> - amount of money offered by a briber</li>
    </ul>
</p>
<p>
    The second one is the number of policemen potential briber knows, which have influence on mayor's decision proportional to:<br/>
    $pc \cdot np^{pp}$, where<br/>
    <ul>
        <li><b>pc</b> - coefficient for the number of known policemen, currently unknown and needs to be estimated</li>
        <li><b>np</b> - number of policemen known by the briber</li>
        <li><b>pp</b> - exponent for the number of policemen, currently unknown and needs to be estimated</li>
    </ul>
</p>
<p>
    Members of the club were also able to collect some observations. Each consists of this observations consists of three features:
    <ul>
        <li><b>money</b> - amount of money offered</li>
        <li><b>policemen</b> - number of policemen potential briber knows</li>
        <li><b>bribe_accepted</b> - binary flag informing if mayor accepted the bribe</li>
    </ul>
</p>
<p>
    Your task is to use TensorFlow to estimate parameter values of a model predicting if the mayor will take a bribe:
    <ul>
        <li><b>mc</b> called <b>v_money_coef</b> below</li>
        <li><b>pc</b> called <b>v_policemen_coef</b> below</li>
        <li><b>pp</b> called <b>v_policemen_pow</b> below</li>
        <li><b>intercept</b> called <b>v_intercept</b> below</li>
    </ul>
</p>

In [ ]:
bribers = read_csv('data/bribers.csv') %>%
    mutate(
        policemen = as.double(policemen)
    )

In [ ]:
# let's take a peek
head(bribers)

Start with loading the data into the graph...

In [ ]:
t_money = NA           # TODO: create a constant with amounts of money offered
t_policemen = NA       # TODO: create a constant with numbers of policemen known
t_bribe_accepted = NA  # TODO: create a constant with flags whether bribes was accepted in given attempts

... and declaring variables for parameters you want to estimate. Set the initial value of each variable to 1.

In [ ]:
v_money_coef = NA        # TODO: create a variable for money_coef (mc). Initialise it with 1.0.

v_policemen_pow = NA     # TODO: create a variable for policemen_power (pp). Initialise it with 1.0.
v_policemen_coef = NA    # TODO: create a variable for policemen_coef (pc). Initialise it with 1.0.

v_intercept = NA         # TODO: create a variable for intercept. Initialise it with 1.0.

Now, you need to build a graph describing, how probability of taking a bribe can be calculated using constants and variables. You also need to define a total loss, and the optimisation process. 

In [ ]:
money_comp = NA     # TODO: calculate money component as money_coef * log(money)
policemen_comp = NA # TODO: calculate policemen component as policemen_coef * (policemen ^ policemen_power)

logit = NA          # TODO: calculate logit as a sum of money component, policemen component and intercept
prob = NA           # TODO: convert logit to a probability using a sigmoid function

loss = NA           # TODO: calculate mean cross-entropy loss of the estimated probability

optimizer = NA      # TODO: create an Adam optimiser with a learning rate equal 0.05
train = NA          # TODO: use optimiser to minimise loss

And now, it's time to create a session, and run the optimisation process:

In [ ]:
sess = tf$Session()
sess$run(tf$global_variables_initializer())

In [ ]:
for(i in 0:1000){
    sess$run(train)
    if(i %% 100 == 0){
        cat(i, ') ', format(Sys.time(), "%Y-%m-%d %X"), ' Loss: ', sess$run(loss), '\n', sep = '')
    }
}

When optimisation is finished, we can read parameter values:

In [ ]:
cat("v_money_coef: ", sess$run(v_money_coef), "\n")
cat("v_policemen_pow: ", sess$run(v_policemen_pow), "\n")
cat("v_policemen_coef: ", sess$run(v_policemen_coef), "\n")
cat("v_intercept: ", sess$run(v_intercept), "\n")

<p>
    Local fortune-teller is sure that these parameters should be equal:<br/>
    <ul>
        <li>money_coef = 1.2</li>
        <li>policemen_pow = 1.3</li>
        <li>policemen_coef = -0.5</li>
        <li>intercept = -3.0</li>
    </ul>
    It seems these numbers are correct, as they often worked. Are your estimates similar?
</p>

# 4. How to check if your model works?

<p>Now let's see how you can check if your model actually works. Generally, the easiest way to do that is to:
    <ol>
        <li>assume some parameters</li>
        <li>generate synthetic data in a way expected by your model using these parameters</li>
        <li>use the model you build to estimate parameters on these artificial data</li>
        <li>compare parameters you originally assumed with these that your model estimated</li>
        <li>improve the model to address issues you observed</li>
    </ol>
</p>

## 4.0 1PL IRT model

<p>
    In this section we will work with the 1-parameter logistic item response theory model (1PL IRT). It is a psychometric model that assumes the probability of giving a correct answer by a student is equal:
</p>
<p>
    $p(correct_{si} = 1) = {1 \over {1+e^{-logit(correct_{si})}}}$<br/>
    $logit(correct_{si} = 1) = skill_s - diff_i$, where
    <ul>
        <li>$p(correct_{si} = 1)$ is a probability that student <i>s</i> answers item <i>i</i> correctly</li>
        <li>$skill_s$ is a skill level of a student <i>s</i></li>
        <li>$diff_i$ is a difficulty level of an item <i>i</i></li>
    </ul>
</p>
<p>
    It means that:
    <ul>
        <li>When student's skill is equal to an item's difficulty the probability of a correct answer is at 50%</li>
        <li>When student's skill is higher than an item's difficulty the probability of a correct answer is higher than 50%. E.g. when this difference is ~3, this probability is at ~95%</li>
        <li>When student's skill is lower than an item's difficulty the probability of a correct answer is lower than 50%. E.g. when this difference is ~-3, this probability is at ~5%</li>
    </ul>
</p>

## 4.1 Parameters

<p>Now, we will draw values of parameters we will use later:</p>

In [ ]:
library(tidyverse)
library(tensorflow)

In [ ]:
item_count = 100
student_count = 1000

items = tibble(
    item = factor(sprintf('item%05d',1:item_count)),
    diff = rnorm(item_count, 0, 1)
)

students = tibble(
    student = factor(sprintf('student%05d', 1:student_count)),
    skill = rnorm(student_count, 0, 1)
)

<p>Let's take a peek</p>

In [ ]:
head(items)
head(students)

## 4.2 Synthetic data generation

Now, we will generate synthetic data doing the simulation based on the 1PL IRT model assumptions:

In [ ]:
full_data = students %>%
    mutate(tmp = 1) %>%               # tmp column is to enable us to do a cross join with items
    inner_join(mutate(items, tmp = 1), by = 'tmp') %>%
    select(-tmp) %>%
    mutate(
        logit = skill - diff,         # we calculate the logit values...
        prob = 1/(1 + exp(-logit)),   # ... convert it to a probabilities ...
        correct = prob >= runif(n())  # ... and now we draw correct and wrong answers using these probabilities
    )
  
test_data = full_data %>%             # test_data has only information we would have in the production setting
    select(student, item, correct) %>%
    mutate(
        student_ind = as.integer(student),
        item_ind = as.integer(item)
    )

In [ ]:
test_data %>% # let's take a peek how these data look like
    sample_n(6)

## 4.3 Use the model for parameter estimation

<p>Before we will use the model, we need to create it:</p>

In [ ]:
# function building a computation graph up to the total loss
build_1pl_irt_model <- function(irt_data){
    # a vector for storing items' difficulties
    t_diffs = tf$Variable(
        tf$random_normal(
           shape = shape(length(levels(irt_data$item))),
           mean = 0,
           stddev = 6.0
        ),
        name = 'diffs'
    )
  
    # a vector for storing students' skill levels
    t_skills = tf$Variable(
        tf$random_normal(
            shape = shape(length(levels(irt_data$student))),
            mean = 0,
            stddev = 6.0
        ),
        name = 'skills'
    )
  
    # vectors for storing difficulties and skill levels related for given observation
    # irt_data$item_ind and irt_data$student_ind are implicitly loaded as constants into the computation graph
    t_diffs_gathered = tf$gather(t_diffs, irt_data$item_ind - 1L)      # we substract 1, because in python indexes start from 0 
    t_skills_gathered = tf$gather(t_skills, irt_data$student_ind - 1L)

    # now we calculate logits and probabilities for each observed data point
    t_logit = t_skills_gathered - t_diffs_gathered
    prob = tf$sigmoid(t_logit)

    # a constant for the observation of a correct or a wrong answer
    y = tf$constant(as.double(irt_data$correct))

    # loss observed in a given row - cross-entropy
    loss_each = tf$losses$log_loss(y, prob)
    # loss averaged over all rows
    loss = tf$reduce_mean(loss_each, name = 'loss')
  
    # we return some of the graph nodes, e.g. vectors with skill levels and difficulties
    list(
        t_diffs = t_diffs,
        t_skills = t_skills,
        t_diffs_gathered = t_diffs_gathered,
        t_skills_gathered = t_skills_gathered,
        prob = prob,
        loss = loss
    )
}

In [ ]:
# function that adds an optimiser and a training node to the graph
add_training = function(m, sess, learning_rate = 0.1){
  optimizer = tf$train$AdamOptimizer(learning_rate)
  train = optimizer$minimize(m$loss)
  
  sess$run(tf$global_variables_initializer())
  
  m$optimizer = optimizer
  m$train = train
  
  m
}

OK, so now when we have functions building a model, we need to call them and build the actual model instance that uses the data we have. Please keep in mind, that because observations are kept in the graph as constants, it needs to be built separately for each dataset we want fit it to.

In [ ]:
model = build_1pl_irt_model(test_data)
sess = tf$Session()
model = add_training(model, sess)

Now, let's do a single step of forward and backward propagation:

In [ ]:
sess$run(model$train)       # each time we run this node, one phase of forward and backward propagation is performed
print(sess$run(model$loss)) # now we can check the value of the loss in the given iteration

And now, let's do a thousand of steps:

In [ ]:
for(i in 0:1000){
    sess$run(model$train)
    if(i %% 100 == 0){
        cat(i, ') ', format(Sys.time(), "%Y-%m-%d %X"), ' Loss: ', sess$run(model$loss), '\n', sep = '')
    }
}

Now, when we fitted model to the data, we can extract the parameters value from the graph:

In [ ]:
# let's extract the estimated difficulty and add it to the items tibble...
items_with_est = cbind(items, est_diff = sess$run(model$t_diffs))
# ... and the estimated skill to the students tibble
students_with_est = cbind(students, est_skill = sess$run(model$t_skills))

## 4.4 Check if these model parameters are estimated correctly

At the start, we can check if estimated values correlated with original ones:

In [ ]:
# now we check if these estimates correlate with 
items_with_est %>%
    summarise(cor(diff, est_diff))

# and check the correlation of original and estimated values
students_with_est %>%
    summarise(cor(skill, est_skill))

Seems, they correlate quite well. Let's check how accurately each of skill levels and difficulties are estimated. We can do that using a scatterplot:

In [ ]:
library(plotly )

In [ ]:
plot_ly() %>%
  add_markers(
    x = ~diff,
    y = ~est_diff,
    data = items_with_est
  )

cat("Mean original difficulty: ", mean(items_with_est$diff), "\n")
cat("Mean estimated difficulty: ", mean(items_with_est$est_diff))

In [ ]:
plot_ly() %>%
  add_markers(
    x = ~skill,
    y = ~est_skill,
    data = students_with_est
  )

cat("Mean original skill: ", mean(students_with_est$skill), "\n")
cat("Mean estimated skill: ", mean(students_with_est$est_skill))

----
We can see that the original and estimated values create a line quite well. But we can also observe a systematic difference between these values. Moreover, it changes each time we rerun our model fitting. To understand what happens we need to look at how our model is constructed:
```
...
t_logit = t_skills_gathered - t_diffs_gathered
...
```

It means that logits would remain the same if we would add or substract the same value from all difficulties and skill levels. Thus, when there is a single best solution: [skills, difficulties], the solution [skills + $\alpha$, difficulties + $\alpha$] is equally as good. Our model just finds one of these equally good solutions. And sometimes it can be something like [skills + 1000, difficulties + 1000]. When we need to fit our model for prediction only it's OK. But when we fit our model for the sake of obtaining interpretable parameters it may pose a problem. To counter it, we may set additional constraints, which will help us select a single solution out of the infinity. For IRT models, in such a situation it's safe to assume that the average skill in the population (or our sample) is equal 0.

## 4.5 Improve the model

<p>
    So, let's modify our model, so it will always look for solutions that will have mean of the skill estimations close to 0.<br/>
    <b>Exercise:</b> replace NA's with operation nodes we need:
</p>

In [ ]:
# function building a computation graph up to the total loss;
# it's similar to the previous one, but contains additional scaling
#   components, which are intended to keep avg. skill at 0
build_1pl_irt_model_scaled <- function(irt_data){
    # a vector for storing items' difficulties
    t_diffs = tf$Variable(
        tf$random_normal(
           shape = shape(length(levels(irt_data$item))),
           mean = 0,
           stddev = 6.0
        ),
        name = 'diffs'
    )
  
    # a vector for storing students' skill levels
    t_skills = tf$Variable(
        tf$random_normal(
            shape = shape(length(levels(irt_data$student))),
            mean = 0,
            stddev = 6.0
        ),
        name = 'skills'
    )
  
    # vectors for storing difficulties and skill levels related for given observation
    # irt_data$item_ind and irt_data$student_ind are implicitly loaded as constants into the computation graph
    t_diffs_gathered = tf$gather(t_diffs, irt_data$item_ind - 1L)      # we substract 1, because in python indexes start from 0 
    t_skills_gathered = tf$gather(t_skills, irt_data$student_ind - 1L)

    # now we calculate logits and probabilities for each observed data point
    t_logit = t_skills_gathered - t_diffs_gathered
    prob = tf$sigmoid(t_logit)

    # a constant for the observation of a correct or a wrong answer
    y = tf$constant(as.double(irt_data$correct))

    # mean of students' skills
    skills_mean = NA # TODO: calculate the mean of t_skills
    
    # loss observed in a given row - cross-entropy
    loss_each = tf$losses$log_loss(y, prob)
    # loss averaged over all rows
    loss = tf$reduce_mean(loss_each, name = 'loss') +
               # it will be lowest when skills_mean will be equal 0
               0.01 * NA # TODO: add the absolute value of the skills_mean to the loss function
  
    # we return some of the graph nodes, e.g. vectors with skill levels and difficulties
    list(
        t_diffs = t_diffs,
        t_skills = t_skills,
        t_diffs_gathered = t_diffs_gathered,
        t_skills_gathered = t_skills_gathered,
        skills_mean = skills_mean,
        prob = prob,
        loss = loss
    )
}

<p>Now, let's build a model for current dataset one more time. This time we will use a function building an improved graph.</p>

In [ ]:
model = build_1pl_irt_model_scaled(test_data)
sess = tf$Session()
model = add_training(model, sess)

Now we do a number of backward and forward propagation steps

In [ ]:
for(i in 0:2000){
    sess$run(model$train)
    if(i %% 100 == 0){
        cat(i, ') ', format(Sys.time(), "%Y-%m-%d %X"), ' Loss: ', sess$run(model$loss), ', Mean skill: ', sess$run(model$skills_mean), '\n', sep = '')
    }
}

Now, we extract estimated parameter values

In [ ]:
items_with_est = cbind(items, est_diff = sess$run(model$t_diffs))
students_with_est = cbind(students, est_skill = sess$run(model$t_skills))

And now, we can compare values estimated by the model with these originally assumed 

In [ ]:
plot_ly() %>%
  add_markers(
    x = ~diff,
    y = ~est_diff,
    data = items_with_est
  )

cat("Mean original difficulty: ", mean(items_with_est$diff), "\n")
cat("Mean estimated difficulty: ", mean(items_with_est$est_diff))

In [ ]:
plot_ly() %>%
  add_markers(
    x = ~skill,
    y = ~est_skill,
    data = students_with_est
  )

cat("Mean original skill: ", mean(students_with_est$skill), "\n")
cat("Mean estimated skill: ", mean(students_with_est$est_skill))

<p>It seems the new model correctly centers skill levels close to 0</p>

## 4.6 Check model with empirical data

In real world, we don't know the true values of parameters. However, it is possible to check if our model works using empirical data.

### 4.6.1 Group estimated probabilities into buckets and compare with CFT (correct on the first try)

Let's prepare data. Observed data are: student id, item id, flag if a given student answer a given item correctly at the first try. Estimated data are: student id, item id, estimated difficulty for a given item, estimated skill for a given student. Based on estimated data we can calculate the probability of the correct answer on the first try.

In [ ]:
# Let's prepare data 
test_estimated_data <- test_data %>%
  select(student, item, correct) %>% 
  inner_join(items_with_est %>% select(-diff), by = 'item') %>% 
  inner_join(students_with_est %>% select(-skill), by = 'student') %>%
  mutate(calc_logit = est_skill - est_diff,
         calc_prob = 1 / (1 + exp(-calc_logit))) # calculate probability of correct answer (CFT)

In [ ]:
head(test_estimated_data)

In [ ]:
# Let's group estimated probabilities into buckets
buckets <- test_estimated_data %>%
  mutate(calc_prob_range = cut(calc_prob, breaks = (0:20)*0.05, include.lowest = TRUE, ordered_result = TRUE))

head(buckets)

<p>For each bucket, we can look at the mean CFT and mean estimated probability of CFT and compare the results. These values for each bucket should be similar.<br>
There is also the accuracy metric provided (the higher accuracy, the better estimation). However, point out that accuracy metric is higher for buckets closer to 0 or 1 but quite low for buckets close to 0.5. Does it seem to be reasonable? </p>

In [ ]:
buckets %>%
  group_by(calc_prob_range) %>%
  summarise(
    n = n(),
    correct_perc = mean(correct),
    correct_perc_calc = mean(calc_prob),
    accuracy = mean((correct & calc_prob > 0.5) | (!correct & calc_prob < 0.5)))

### 4.6.2 Calculate cross-entropy between estimation and observed value, compare results to baselines

Let's prepare baselines. We calculate mean CFT for each item, mean CFT for each student, and mean CFT for a whole book.

In [ ]:
head(test_estimated_data)

In [ ]:
baselines <- test_estimated_data %>% 
  group_by(item) %>% 
  mutate(item_mean_correct = mean(correct)) %>% 
  group_by(student) %>% 
  mutate(student_mean_correct = mean(correct)) %>% 
  ungroup() %>% 
  mutate(book_mean_correct = mean(correct))

head(baselines)

Now we can calculate cross entropy for observed values and our estimation from 1PL model and for observed values and our baselines.

In [ ]:
sess <- tf$Session()

# setting nodes
tf_correct <- tf$constant(baselines$correct %>% as.numeric())

tf_calc_prob <- tf$constant(baselines$calc_prob)
tf_item_mean_correct <- tf$constant(baselines$item_mean_correct)
tf_student_mean_correct <- tf$constant(baselines$student_mean_correct)
tf_book_mean_correct <- tf$constant(baselines$book_mean_correct)
tf_coin_flip <- tf$fill(list(tf$size(tf_correct)), tf$constant(0.5))

# cross-entropy between tf_correct and estimations
ce_calc_prob <- tf$losses$log_loss(labels = tf_correct, predictions = tf_calc_prob)
ce_item_mean_correct  <- tf$losses$log_loss(labels = tf_correct, predictions = tf_item_mean_correct)
ce_student_mean_correct <- tf$losses$log_loss(labels = tf_correct, predictions = tf_student_mean_correct)
ce_book_mean_correct <- tf$losses$log_loss(labels = tf_correct, predictions = tf_book_mean_correct)
ce_coin_flip <- tf$losses$log_loss(labels = tf_correct, predictions = tf_coin_flip)

The best model (with the lowest cross entropy) should be model 1PL as it takes into account both: students' skills and items' difficulties. Mean CFT for each item and mean CFT for each student should have higher cross entropy than 1PL. The worst 'model' should be a coin flip.

In [ ]:
cat(sprintf('Cross entropy for observed values compared with:
- values estimated by 1PL model: %f, 
- mean CFT of each item: %f, 
- mean CFT of each student: %f, 
- mean CFT of whole book: %f, 
- coin flip: %f', 
sess$run(ce_calc_prob), sess$run(ce_item_mean_correct), sess$run(ce_student_mean_correct), 
sess$run(ce_book_mean_correct), sess$run(ce_coin_flip)))

# 5. Further reading

## 5.1 Eager execution in TensorFlow

Alternatively to the logic described above, a new mode called the _eager execution_ was recently introduced to the TensorFlow environment. Using this mode, we don't need to run computations in sessions. Instead, all calculations are done immediately. Working in this mode is more comfortable for experimenting, e.g. it lets debug code much more easily, but it's less efficient than a session-based mode when we put such code on production. To read more about the _eager execution_ mode you can go to:

- [8 Things To Do Differently in Tensorflow’s Eager Execution Mode](https://medium.com/coinmonks/8-things-to-do-differently-in-tensorflows-eager-execution-mode-47cf429aa3ad)
- [Eager Execution](https://www.tensorflow.org/guide/eager) - TensorFlow documentation
- [More flexible models with TensorFlow eager execution and Keras](https://blogs.rstudio.com/tensorflow/posts/2018-10-02-eager-wrapup/) - TensorFlow for R Blog


## 5.2 Probabilistic programming tools

<p>Probabilistic programming tools let us easily estimate parameter values not only as most likely point estimates, but as so-called posterior distributions. These distributions let us for example find out how accurate our estimates are.</p>
<p>E.g. let's assume that MLE of a parameter is equal 2.0. Having only that information, we don't know how accurate it is. But, when we have a distribution of a parameter likelihood, we can check the accuracy of the estimation, (e.g. how much information we were able to extract from the data). When an interval containing 95% of most likely parameter values is narrow, e.g $[1.99; 2.01]$, then we know our estimates are precise. But, when it is very wide, e.g. $[-98; 102]$, we know it isn't accurate.</p>

- [greta](https://greta-stats.org) - a package that lets us specify a statistical model directly in the R code. It's built on top of TensorFlow, so it can seamlessly use a GPGPU when it's available in the system 